In [3]:
# For CUDA 11.x
!pip install cupy-cuda11x

# For CUDA 12.x  
!pip install cupy-cuda12x

# Alternative: Install based on your CUDA version
!pip install cupy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/77.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/77.1 MB ? eta -:--:--
   ---------------------------------------- 0.2/77.1 MB 3.1 MB/s eta 0:00:25
   ---------------------------------------- 0.4/77.1 MB 4.4 MB/s eta 0:00:18
   ---------------------------------------- 0.7/77.1 MB 4.3 MB/s eta 0:00:18
   ---------------------------------------- 0.9/77.1 MB 4.6 MB/s eta 0:00:17
    --------------------------------------- 1.0/77.1 MB 4.7 MB/s eta 0:00:17
    --------------------------------------- 1.0/77.1 MB 4.7 MB/s eta 0:00:17
    --------------------------------------- 1.0/77.1 MB 4.7 MB/s eta 0:00:17
    --------------------------------------- 1.0/77.1 MB 4.7 MB/s eta 0:00:17
    --------------------------------------- 1.0/77.1 MB 4.7 MB/s eta 0:00:17
    --------------------------------------- 1.2/77.1 MB 2.6 MB/s eta 0:00:29
    --------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/82.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/82.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/82.2 MB 825.8 kB/s eta 0:01:40
   ---------------------------------------- 0.4/82.2 MB 3.5 MB/s eta 0:00:24
   ---------------------------------------- 0.6/82.2 MB 3.7 MB/s eta 0:00:23
   ---------------------------------------- 0.7/82.2 MB 3.4 MB/s eta 0:00:25
   ---------------------------------------- 0.8/82.2 MB 3.1 MB/s eta 0:00:27
   ---------------------------------------- 0.8/82.2 MB 3.0 MB/s eta 0:00:28
   ---------------------------------------- 0.8/82.2 MB 3.0 MB/s eta 0:00:28
   ---------------------------------------- 0.8/82.2 MB 3.0 MB/s eta 0:00:28
   ---------------------------------------- 0.8/82.2 MB 3.0 MB/s eta 0:00:28
   ---------------------------------------- 0.8/82.2 MB 3.0 MB/s eta 0:00:28
   -------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.5 MB 393.8 kB/s eta 0:00:09
     ---- ----------------------------------- 0.4/3.5 MB 3.0 MB/s eta 0:00:02
     ---------- ----------------------------- 0.9/3.5 MB 5.2 MB/s eta 0:00:01
     --------------- ------------------------ 1.4/3.5 MB 6.3 MB/s eta 0:00:01
     --------------------- ------------------ 1.8/3.5 MB 6.9 MB/s eta 0:00:01
     -------------------------- ------------- 2.4/3.5 MB 7.5 MB/s eta 0:00:01
     -------------------------------- ------- 2.8/3.5 MB 7.9 MB/s eta 0:00:01
     -------------------------------------- - 3.3/3.5 MB 8.2 MB/s eta 0:00:01
     ---------------------------------------- 3.5/3.5 MB 7.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: fi

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [69 lines of output]
      Generating cache key from header files...
      Cache key (1729 files matching C:\Users\Milon\AppData\Local\Temp\pip-install-_ohrrgk4\cupy_b18cb93096824768a438a71d25b84e50\cupy\_core\include\**): 62426478e3e7017e0abfdd71b0667fdffa294302
      Clearing directory: C:\Users\Milon\AppData\Local\Temp\pip-install-_ohrrgk4\cupy_b18cb93096824768a438a71d25b84e50\cupy\.data
      Looking for NVTX: C:\Program Files\NVIDIA Corporation\Nsight Systems *\target-windows-x64\nvtx
      Using NVTX at: C:\Program Files\NVIDIA Corporation\Nsight Systems 2022.4.2\target-windows-x64\nvtx
      
      -------- Configuring Module: cuda --------
      Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      **************************************************
 

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
import warnings
warnings.filterwarnings('ignore')

# GPU acceleration imports
try:
    import cupy as cp
    import cupyx.scipy.signal as cupy_signal
    GPU_AVAILABLE = True
    print("GPU acceleration available with CuPy")
except ImportError:
    print("CuPy not available. Install with: pip install cupy-cuda11x (or cupy-cuda12x)")
    GPU_AVAILABLE = False
    cp = np

try:
    import torch
    import torch.nn.functional as F
    TORCH_AVAILABLE = torch.cuda.is_available() if 'torch' in locals() else False
    if TORCH_AVAILABLE:
        print(f"PyTorch GPU available: {torch.cuda.get_device_name()}")
except ImportError:
    TORCH_AVAILABLE = False
    print("PyTorch not available for GPU acceleration")

class GPUEEGScalogramGenerator:
    def __init__(self, sz_input_dir, hc_input_dir, output_dir, sampling_rate=250, use_gpu=True):
        """
        Initialize the GPU-accelerated EEG Scalogram Generator using Continuous Wavelet Transform (CWT)
        
        Parameters:
        sz_input_dir (str): Directory containing CSV files with Schizophrenia EEG data
        hc_input_dir (str): Directory containing CSV files with Healthy Control EEG data
        output_dir (str): Root directory to save scalogram images
        sampling_rate (int): EEG sampling rate in Hz (default: 250 Hz)
        use_gpu (bool): Whether to use GPU acceleration
        """
        self.sz_input_dir = sz_input_dir
        self.hc_input_dir = hc_input_dir
        self.output_dir = output_dir
        self.sampling_rate = sampling_rate
        self.segment_duration = 3  # 3 seconds
        self.segment_samples = self.sampling_rate * self.segment_duration  # 750 samples for 3 seconds
        
        # GPU settings
        self.use_gpu = use_gpu and GPU_AVAILABLE
        if self.use_gpu:
            self.xp = cp  # Use CuPy for array operations
            print(f"Using GPU: {cp.cuda.get_device_name()}")
        else:
            self.xp = np  # Use NumPy for CPU operations
            print("Using CPU (GPU not available or disabled)")
        
        # Wavelet parameters
        self.freq_min = 0.5  # Minimum frequency (Hz)
        self.freq_max = 45   # Maximum frequency (Hz)
        self.num_scales = 100  # Number of scales/frequencies
        
        # Create frequency array
        self.frequencies = np.logspace(np.log10(self.freq_min), np.log10(self.freq_max), self.num_scales)
        
        # Pre-compute Morlet wavelets for GPU
        self._precompute_wavelets()
        
        # Create output directories
        self.sz_dir = os.path.join(output_dir, 'Schizophrenia')
        self.hc_dir = os.path.join(output_dir, 'HealthyControl')
        os.makedirs(self.sz_dir, exist_ok=True)
        os.makedirs(self.hc_dir, exist_ok=True)
        
    def _precompute_wavelets(self):
        """Pre-compute Morlet wavelets for all frequencies"""
        print("Pre-computing Morlet wavelets...")
        
        # Morlet wavelet parameters
        fb = 1.5  # Bandwidth parameter
        fc = 1.0  # Center frequency
        
        # Time array for wavelet (make it long enough for lowest frequency)
        max_period = 1.0 / self.freq_min
        wavelet_length = int(8 * max_period * self.sampling_rate)  # 8 periods
        if wavelet_length % 2 == 0:
            wavelet_length += 1  # Make it odd
            
        t = np.arange(-wavelet_length//2, wavelet_length//2 + 1) / self.sampling_rate
        
        # Pre-compute all wavelets
        self.wavelets = []
        self.wavelet_length = wavelet_length
        
        for freq in self.frequencies:
            # Scale time for this frequency
            scaled_t = 2 * np.pi * freq * t
            
            # Complex Morlet wavelet
            sigma = fb / (2 * np.pi * freq)  # Standard deviation
            wavelet = (1 / np.sqrt(np.pi * fb)) * np.exp(1j * fc * scaled_t) * np.exp(-scaled_t**2 / (2 * fb))
            
            # Normalize
            wavelet = wavelet / np.sqrt(np.sum(np.abs(wavelet)**2))
            
            if self.use_gpu:
                wavelet = cp.asarray(wavelet)
                
            self.wavelets.append(wavelet)
        
        print(f"Pre-computed {len(self.wavelets)} wavelets")
    
    def bandpass_filter(self, data, lowcut=0.5, highcut=45, order=4):
        """
        Apply bandpass filter to EEG data (CPU-based as it's fast enough)
        
        Parameters:
        data (array): EEG signal data
        lowcut (float): Low cutoff frequency
        highcut (float): High cutoff frequency
        order (int): Filter order
        """
        nyquist = 0.5 * self.sampling_rate
        low = lowcut / nyquist
        high = highcut / nyquist
        b, a = butter(order, [low, high], btype='band')
        
        # Keep filtering on CPU as it's already optimized
        if self.use_gpu and hasattr(data, 'get'):
            data_cpu = data.get()  # Convert to CPU
            filtered_data = filtfilt(b, a, data_cpu, axis=0)
            return cp.asarray(filtered_data)  # Convert back to GPU
        else:
            return filtfilt(b, a, data, axis=0)
    
    def gpu_cwt_fast(self, signal):
        """
        Fast GPU-based Continuous Wavelet Transform using pre-computed wavelets
        
        Parameters:
        signal (array): 1D EEG signal data
        
        Returns:
        array: CWT coefficients (power in dB)
        """
        if self.use_gpu:
            signal_gpu = cp.asarray(signal) if not isinstance(signal, cp.ndarray) else signal
        else:
            signal_gpu = signal
            
        # Pad signal for convolution
        pad_length = self.wavelet_length // 2
        if self.use_gpu:
            signal_padded = cp.pad(signal_gpu, pad_length, mode='symmetric')
        else:
            signal_padded = np.pad(signal_gpu, pad_length, mode='symmetric')
        
        # Compute CWT using convolution for all frequencies
        cwt_matrix = []
        
        for i, wavelet in enumerate(self.wavelets):
            # Convolve signal with wavelet (this is the main computational bottleneck)
            if self.use_gpu:
                # Use CuPy's optimized convolution
                conv_result = cp.convolve(signal_padded, cp.conj(wavelet[::-1]), mode='same')
                # Extract the valid part
                conv_result = conv_result[pad_length:-pad_length]
            else:
                conv_result = np.convolve(signal_padded, np.conj(wavelet[::-1]), mode='same')
                conv_result = conv_result[pad_length:-pad_length]
            
            cwt_matrix.append(conv_result)
        
        # Stack all frequency results
        if self.use_gpu:
            cwt_matrix = cp.stack(cwt_matrix)
        else:
            cwt_matrix = np.stack(cwt_matrix)
        
        # Convert to power and dB scale
        power = self.xp.abs(cwt_matrix) ** 2
        power_db = 10 * self.xp.log10(power + 1e-12)
        
        return power_db
    
    def gpu_batch_cwt(self, segment):
        """
        Compute CWT for all channels in a segment using GPU batch processing
        
        Parameters:
        segment (array): EEG segment with shape (samples, channels)
        
        Returns:
        array: Average scalogram across all channels
        """
        n_samples, n_channels = segment.shape
        
        if self.use_gpu:
            segment_gpu = cp.asarray(segment)
        else:
            segment_gpu = segment
        
        # Process all channels
        all_scalograms = []
        
        for ch in range(n_channels):
            if self.use_gpu:
                signal = segment_gpu[:, ch]
            else:
                signal = segment_gpu[:, ch]
                
            # Compute CWT for this channel
            scalogram = self.gpu_cwt_fast(signal)
            all_scalograms.append(scalogram)
        
        # Stack and average all scalograms
        if self.use_gpu:
            all_scalograms = cp.stack(all_scalograms)
            avg_scalogram = cp.mean(all_scalograms, axis=0)
            # Convert back to CPU for plotting
            avg_scalogram = avg_scalogram.get()
        else:
            all_scalograms = np.stack(all_scalograms)
            avg_scalogram = np.mean(all_scalograms, axis=0)
        
        return avg_scalogram
    
    def segment_signal(self, data):
        """
        Segment EEG signal into 3-second windows
        
        Parameters:
        data (array): EEG data with shape (samples, channels)
        
        Returns:
        list: List of 3-second segments
        """
        segments = []
        n_samples, n_channels = data.shape
        
        # Calculate number of complete 3-second segments
        n_segments = n_samples // self.segment_samples
        
        for i in range(n_segments):
            start_idx = i * self.segment_samples
            end_idx = start_idx + self.segment_samples
            segment = data[start_idx:end_idx, :]
            segments.append(segment)
            
        return segments
    
    def create_combined_scalogram(self, segment, patient_id, segment_idx, label):
        """
        Create a single scalogram image by averaging across all 19 channels using GPU acceleration
        
        Parameters:
        segment (array): 3-second EEG segment with shape (samples, channels)
        patient_id (str): Patient identifier
        segment_idx (int): Segment index
        label (str): 'SZ' for Schizophrenia or 'HC' for Healthy Control
        """
        # Create time axis
        time_axis = np.linspace(0, self.segment_duration, segment.shape[0])
        
        # Compute GPU-accelerated CWT for all channels and average
        print(f"  Computing GPU-accelerated CWT for all {segment.shape[1]} channels...")
        avg_scalogram = self.gpu_batch_cwt(segment)
        print(f"  Completed GPU CWT and averaging")
        
        # Create and save the averaged scalogram
        plt.figure(figsize=(12, 8))
        
        # Create main plot
        ax = plt.gca()
        im = ax.imshow(
            avg_scalogram, 
            aspect='auto', 
            origin='lower',
            extent=[time_axis[0], time_axis[-1], self.frequencies[0], self.frequencies[-1]],
            cmap='viridis'
        )
        
        # Add colorbar on the right side
        plt.subplots_adjust(right=0.85)
        cbar_ax = plt.gcf().add_axes([0.87, 0.15, 0.03, 0.7])
        plt.colorbar(im, cax=cbar_ax, label='Power (dB)')
        
        ax.set_title(f'GPU-Accelerated Average EEG Scalogram - {label} Patient {patient_id} - Segment {segment_idx}', 
                 fontsize=14, fontweight='bold')
        ax.set_xlabel('Time (s)', fontsize=12)
        ax.set_ylabel('Frequency (Hz)', fontsize=12)
        ax.set_yscale('log')
        ax.set_ylim(self.freq_min, self.freq_max)
        
        # Save the scalogram image
        if label == 'SZ':
            save_dir = self.sz_dir
        else:
            save_dir = self.hc_dir
            
        filename = f'{label}_patient_{patient_id}_segment_{segment_idx:03d}_scalogram_gpu.png'
        filepath = os.path.join(save_dir, filename)
        
        plt.savefig(filepath, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close()
        
        return filepath
    
    def process_patient_file(self, csv_file, patient_id, label):
        """
        Process a single patient's EEG CSV file using GPU acceleration
        
        Parameters:
        csv_file (str): Path to CSV file
        patient_id (str): Patient identifier
        label (str): 'SZ' for Schizophrenia or 'HC' for Healthy Control
        """
        print(f"Processing {label} Patient {patient_id} with GPU acceleration...")
        
        try:
            # Load EEG data
            data = pd.read_csv(csv_file)
            
            # Convert to numpy array
            eeg_data = data.values
            
            # Ensure we have the correct number of channels
            if eeg_data.shape[1] != 19:
                print(f"Warning: Expected 19 channels, got {eeg_data.shape[1]} for patient {patient_id}")
                if eeg_data.shape[1] > 19:
                    eeg_data = eeg_data[:, :19]
                else:
                    padding = np.zeros((eeg_data.shape[0], 19 - eeg_data.shape[1]))
                    eeg_data = np.hstack([eeg_data, padding])
            
            print(f"  EEG data shape: {eeg_data.shape}")
            
            # Apply bandpass filter
            print(f"  Applying bandpass filter ({self.freq_min}-{self.freq_max} Hz)...")
            filtered_data = self.bandpass_filter(eeg_data)
            
            # Segment the data into 3-second windows
            print(f"  Segmenting data into {self.segment_duration}-second windows...")
            segments = self.segment_signal(filtered_data)
            
            print(f"  Generated {len(segments)} segments from patient {patient_id}")
            
            # Create GPU-accelerated scalogram images for each segment
            saved_files = []
            for idx, segment in enumerate(segments):
                print(f"  Processing segment {idx + 1}/{len(segments)} with GPU...")
                filepath = self.create_combined_scalogram(segment, patient_id, idx, label)
                saved_files.append(filepath)
                print(f"  Saved: {os.path.basename(filepath)}")
            
            return saved_files
            
        except Exception as e:
            print(f"Error processing patient {patient_id}: {str(e)}")
            return []
    
    def get_patient_files(self, directory):
        """Get all CSV files from a directory"""
        csv_files = []
        if os.path.exists(directory):
            for file in os.listdir(directory):
                if file.lower().endswith('.csv'):
                    csv_files.append(os.path.join(directory, file))
            csv_files.sort()
        else:
            print(f"Warning: Directory {directory} does not exist!")
        
        return csv_files
    
    def process_all_patients(self):
        """Process all patients using GPU acceleration"""
        print("=== GPU-Accelerated EEG Scalogram Generation ===")
        print(f"GPU Status: {'Enabled' if self.use_gpu else 'Disabled'}")
        if self.use_gpu:
            print(f"GPU Memory: {cp.cuda.Device().mem_info[1] / 1024**3:.1f} GB")
        print("Generating combined scalograms averaged across all 19 channels\n")
        
        # Get patient files
        sz_patient_files = self.get_patient_files(self.sz_input_dir)
        hc_patient_files = self.get_patient_files(self.hc_input_dir)
        
        print(f"Found {len(sz_patient_files)} Schizophrenia patient files")
        print(f"Found {len(hc_patient_files)} Healthy Control patient files")
        
        if len(sz_patient_files) == 0 and len(hc_patient_files) == 0:
            print("No CSV files found in the specified directories!")
            return
        
        total_sz_images = 0
        total_hc_images = 0
        
        # Process with timing
        import time
        start_time = time.time()
        
        print("\n=== Processing Schizophrenia Patients ===")
        for i, csv_file in enumerate(sz_patient_files, 1):
            patient_id = f"SZ_{i:02d}"
            patient_start = time.time()
            saved_files = self.process_patient_file(csv_file, patient_id, 'SZ')
            patient_time = time.time() - patient_start
            total_sz_images += len(saved_files)
            print(f"  Generated {len(saved_files)} scalograms for {patient_id} in {patient_time:.2f}s\n")
        
        print("=== Processing Healthy Control Patients ===")
        for i, csv_file in enumerate(hc_patient_files, 1):
            patient_id = f"HC_{i:02d}"
            patient_start = time.time()
            saved_files = self.process_patient_file(csv_file, patient_id, 'HC')
            patient_time = time.time() - patient_start
            total_hc_images += len(saved_files)
            print(f"  Generated {len(saved_files)} scalograms for {patient_id} in {patient_time:.2f}s\n")
        
        total_time = time.time() - start_time
        
        print("=== Summary ===")
        print(f"Total processing time: {total_time:.2f} seconds")
        print(f"Average time per scalogram: {total_time/(total_sz_images + total_hc_images):.3f}s")
        print(f"Total Schizophrenia scalograms: {total_sz_images}")
        print(f"Total Healthy Control scalograms: {total_hc_images}")
        print(f"Total scalograms generated: {total_sz_images + total_hc_images}")
        print(f"GPU acceleration: {'Enabled' if self.use_gpu else 'Disabled'}")

# Example usage
if __name__ == "__main__":
    # Check GPU availability
    print("=== GPU Setup Check ===")
    if GPU_AVAILABLE:
        print(f"CuPy available - GPU: {cp.cuda.get_device_name()}")
        print(f"GPU Memory: {cp.cuda.Device().mem_info[1] / 1024**3:.1f} GB total")
    else:
        print("CuPy not available - will use CPU")
        print("To install CuPy for CUDA 11.x: pip install cupy-cuda11x")
        print("To install CuPy for CUDA 12.x: pip install cupy-cuda12x")
    
    # Define paths
    sz_input_directory = "D:/Milon2/SD14/schizophrenia"
    hc_input_directory = "D:/Milon2/SD14/healthy"
    output_directory = "D:/Milon2/scalogram/images"
    
    # Create GPU-accelerated scalogram generator
    generator = GPUEEGScalogramGenerator(
        sz_input_dir=sz_input_directory,
        hc_input_dir=hc_input_directory,
        output_dir=output_directory,
        sampling_rate=256,
        use_gpu=True  # Set to False to use CPU
    )
    
    # Process all patients with GPU acceleration
    print("\nStarting GPU-accelerated scalogram generation...")
    generator.process_all_patients()
    
    print("\nGPU-accelerated scalogram generation completed!")
    print(f"Check the output directory: {output_directory}")
    print("Each scalogram represents GPU-computed average across all 19 EEG channels")

CuPy not available. Install with: pip install cupy-cuda11x (or cupy-cuda12x)
PyTorch not available for GPU acceleration
=== GPU Setup Check ===
CuPy not available - will use CPU
To install CuPy for CUDA 11.x: pip install cupy-cuda11x
To install CuPy for CUDA 12.x: pip install cupy-cuda12x
Using CPU (GPU not available or disabled)
Pre-computing Morlet wavelets...
Pre-computed 100 wavelets

Starting GPU-accelerated scalogram generation...
=== GPU-Accelerated EEG Scalogram Generation ===
GPU Status: Disabled
Generating combined scalograms averaged across all 19 channels

Found 14 Schizophrenia patient files
Found 14 Healthy Control patient files

=== Processing Schizophrenia Patients ===
Processing SZ Patient SZ_01 with GPU acceleration...
  EEG data shape: (211250, 19)
  Applying bandpass filter (0.5-45 Hz)...
  Segmenting data into 3-second windows...
  Generated 275 segments from patient SZ_01
  Processing segment 1/275 with GPU...
  Computing GPU-accelerated CWT for all 19 channels...

KeyboardInterrupt: 